# Backtest the strategies

Use an LLM to go through and predict the buy/ sell/ hold recommendation for the company for the given date. Steps needed:

1. Load the LLM - use DeepSeek R1 Qwen model at 7B parameters first and try the quantised models next
2. Step through each data and each financial statement to get a result
3. Log the results in a file and save to S3 (will need a logging file to save to S3 and resume in case of kernel crash)
4. Need a backtesting framework to apply the results


## Load libraries needed

In [8]:
import json
import boto3
from s3fs import S3FileSystem
import os

import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
import torch
from accelerate import Accelerator

import pandas as pd

from IPython.display import Markdown, display

from helper import get_s3_folder
import s3_model
import company_data
from s3_model import S3ModelHelper

In [9]:
import importlib
importlib.reload(company_data)

<module 'company_data' from '/project/company_data.py'>

## Load the LLM

Models to test:
- Qwen (Qwen/Qwen2.5-7B-Instruct)
- Llama (meta-llama/Llama-3.2-7B-Instruct)
- DeepSeek (deepseek-ai/DeepSeek-R1-Distill-Qwen-14B)

In [10]:
# Log into Huggingface

with open('pass.txt') as p:
    hf_login = p.read()
    
hf_login = hf_login[hf_login.find('=')+1:hf_login.find('\n')]
login(hf_login, add_to_git_credential=False)

In [10]:
c = s3_model.S3ModelHelper(s3_sub_folder='tmp/fs')
c.clear_folder('deepseek')

In [11]:
# Flag to download from Huggingface again or use stored model
USE_HF = False

model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"
model_id_s3 = 'deepseek'


if USE_HF:
   
    pipeline = transformers.pipeline(
        "text-generation",
        model=model_id,
        model_kwargs={"torch_dtype": torch.bfloat16},
        device_map="auto",
    )

    model = AutoModelForCausalLM.from_pretrained(model_id, device_map='auto', torch_dtype=torch.bfloat16 )
    tokenizer = AutoTokenizer.from_pretrained(model_id)
else:
    model_helper = s3_model.S3ModelHelper(s3_sub_folder='tmp/fs')
    model = model_helper.load_model(model_id_s3)
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        model_kwargs={"torch_dtype": torch.bfloat16},
        device_map="auto",
    )
    model_helper.clear_folder(model_id_s3)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Device set to use cuda:0


## Load Financial PIT dataset

In [3]:
## Load from S3 using the helper file
sec_helper = company_data.SecurityData('tmp/fs','data_quarterly_pit.json')
all_data = sec_helper.get_all_data()

In [65]:
# USE WHILE DEVELOPING to
importlib.reload(company_data)
sec_helper = company_data.SecurityData('tmp/fs','data_quarterly_pit.json', all_data)

In [66]:
sec_helper.get_security_statement('2020-01-31','AON UN Equity','px')

,Price
Date,
2019-01-31,100.83
2019-02-28,101.25
2019-03-31,103.69
2019-04-30,118.13
2019-05-31,124.87
2019-06-30,127.68
2019-07-31,127.12
2019-08-31,129.44
2019-09-30,124.43


In [5]:
system_prompt = "You are a financial analyst and must make a buy, sell or hold decision on a company based only on the provided datasets. \
        Compute common financial ratios and then determine the buy sell decision. Explain your reasons and answer in a format that compiles to a JSON object.\
        Answer as a JSON string with the following example format: \
        {'Investment Decision': BUY, 'Reason': 'Gross profit and EPS have both increased over time'}"


In [6]:
prompt = sec_helper.get_prompt('2020-01-31','AON UN Equity', system_prompt)

## Run an example in LLM

Run into out of memory problem - Potential fixes:
1. reduce size of model (quantize)
2. explore multi-gpu
3. reduce tokens

https://saturncloud.io/blog/how-to-solve-cuda-out-of-memory-error-in-pytorch/

https://huggingface.co/docs/accelerate/usage_guides/distributed_inference

https://medium.com/@geronimo7/llms-multi-gpu-inference-with-accelerate-5a8333e4c5db

Problem with splitting a single prompt into multiple gpus to calculate the result. Tensor parallelism - https://huggingface.co/docs/transformers/main/en/perf_train_gpu_many#tensor-parallelism

nvidia-smi will show available GPUs on the system.

In [76]:
accelerator = Accelerator()

In [77]:
a_model = accelerator.prepare(model)

In [72]:
formatted_chat = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)
outputs = pipeline(
    prompt,
    max_new_tokens=200,
)

test_output = outputs[0]['generated_text'][-1]
display(Markdown(test_output['content']))

OutOfMemoryError: CUDA out of memory. Tried to allocate 150.00 MiB. GPU 1 has a total capacty of 21.99 GiB of which 79.00 MiB is free. Process 10911 has 21.90 GiB memory in use. Of the allocated memory 21.64 GiB is allocated by PyTorch, and 18.71 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [78]:
from accelerate import Accelerator
from accelerate.utils import gather_object

accelerator = Accelerator()

# each GPU creates a string
message=[ f"Hello this is GPU {accelerator.process_index}" ] 

# collect the messages from all GPUs
messages=gather_object(message)

# output the messages only on the main process with accelerator.print() 
accelerator.print(messages)

['Hello this is GPU 0']


In [81]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved

In [82]:
f

1251859968

In [83]:
torch.cuda.mem_get_info()

(124846080, 23609475072)

In [84]:
torch.cuda.empty_cache()